In [60]:
import os
from dotenv import load_dotenv
load_dotenv()



chatgroqapi=os.getenv('GROQ_API_KEY')

In [61]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="Llama3-8b-8192",api_key=chatgroqapi)

In [62]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
@tool
def Multiply(a:int,b:int)->int:
    """Given 2 numbers a and b this tool returns their product"""
    return (a*b)

Multiply.invoke({'a':7,"b":4})

28

# tool binding

In [63]:
llmwithtool=llm.bind_tools([Multiply])
llmwithtool.invoke("HI")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hzma', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 901, 'total_tokens': 973, 'completion_time': 0.06, 'prompt_time': 0.112380192, 'queue_time': 0.25149582600000003, 'total_time': 0.172380192}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a76d4e1e-e818-43e5-b209-59b04accb352-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_hzma', 'type': 'tool_call'}], usage_metadata={'input_tokens': 901, 'output_tokens': 72, 'total_tokens': 973})

In [64]:
result=llmwithtool.invoke("multiply 4 with 10")

In [65]:
result.tool_calls[0]

{'name': 'Multiply',
 'args': {'a': 4, 'b': 10},
 'id': 'call_r262',
 'type': 'tool_call'}

In [66]:
Multiply.invoke(result.tool_calls[0])

ToolMessage(content='40', name='Multiply', tool_call_id='call_r262')

In [73]:

query=HumanMessage("can you multiply 3 with 1000")
messages=[query]

messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [74]:
result=llmwithtool.invoke(messages)

In [75]:
messages.append(result)

messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wtv4', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 909, 'total_tokens': 982, 'completion_time': 0.060833333, 'prompt_time': 0.169489593, 'queue_time': 1.312696695, 'total_time': 0.230322926}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0e173c74-783d-4b37-996d-d7db96ad4944-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_wtv4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 909, 'output_tokens': 73, 'total_tokens': 982})]

In [107]:
tool_result=Multiply.invoke(result.tool_calls[0])
tool_result

ToolMessage(content='3000', name='Multiply', tool_call_id='call_wtv4')

In [77]:
messages.append(tool_result)

messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wtv4', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 909, 'total_tokens': 982, 'completion_time': 0.060833333, 'prompt_time': 0.169489593, 'queue_time': 1.312696695, 'total_time': 0.230322926}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--0e173c74-783d-4b37-996d-d7db96ad4944-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_wtv4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 909, 'output_tokens': 73, 'total_tokens': 982}),
 ToolMessage(content='3000', name='Multiply', tool_call_id='call_wtv4')]

In [80]:
llmwithtool.invoke(messages)

AIMessage(content='3000', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 1975, 'total_tokens': 1978, 'completion_time': 0.0025, 'prompt_time': 0.245958421, 'queue_time': -0.27180486000000004, 'total_time': 0.248458421}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run--869128ee-58fe-44cb-b2d7-ae37bcf65626-0', usage_metadata={'input_tokens': 1975, 'output_tokens': 3, 'total_tokens': 1978})

## Currency conversion

In [82]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import requests

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [83]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [84]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1748044801,
 'time_last_update_utc': 'Sat, 24 May 2025 00:00:01 +0000',
 'time_next_update_unix': 1748131201,
 'time_next_update_utc': 'Sun, 25 May 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4133}

In [85]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [96]:
llm=ChatGroq(model="Llama3-8b-8192",api_key=chatgroqapi)
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [97]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [98]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [99]:
ai_message = llm_with_tools.invoke(messages)

In [100]:
messages.append(ai_message)

In [101]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_dkx6',
  'type': 'tool_call'}]

In [102]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [103]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dkx6', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 1048, 'total_tokens': 1125, 'completion_time': 0.064166667, 'prompt_time': 0.13094705, 'queue_time': 0.24929977999999997, 'total_time': 0.195113717}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--74f58998-e11a-446c-bc78-aec8167d0f48-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'call_dkx6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1048, 'output_tokens': 77, 'total_tokens':

In [105]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5d4e', 'function': {'arguments': '{"base_currency_value":10,"function":{"name":"get_conversion_factor","result":{"conversion_rate":0.01171}}}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 1255, 'total_tokens': 1308, 'completion_time': 0.044166667, 'prompt_time': 0.157872051, 'queue_time': 0.25435510299999997, 'total_time': 0.202038718}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6560e43f-4a2c-4071-9d52-c3940623e62b-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 10, 'function': {'name': 'get_conversion_factor', 'result': {'conversion_rate': 0.01171}}}, 'id': 'call_5d4e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1255, 'output_tokens': 53, 'total_tokens': 1308})